In [1]:
FS = 625_000
LOWCUT = 30_000.0
HIGHCUT = 40_000.0

In [2]:
def match_times(times, dt):
    '''
    Takes a 2d list of timings and groups the times together if they are within dt of each other. If one channel does not have a corresponding time, it is filled with None.
    each channel has the times in sequential order (sorted).
    '''
    groupings = []
    indices = [0] * len(times)
    while True:
        ind_times = [times[i][indices[i]] 
                        if indices[i] < len(times[i])
                        else float('inf') 
                    for i in range(len(times))]
        
        min_time = min(ind_times)
        
        group = [None]*len(times)
        
        for i in range(len(times)):
            if indices[i] == len(times[i]):
                continue
            if abs(times[i][indices[i]] - min_time) < dt:
                group[i] = times[i][indices[i]]
                indices[i] += 1
        groupings.append(group)

        if all([indices[i] == len(times[i]) for i in range(len(times))]):
            break
        
    return groupings
    

In [3]:
def what_comes_first(groupings):
    firsts = []
    for group in groupings:
        min_val = min([x for x in group if x is not None])
        firsts.append(group.index(min_val))
    return firsts
        

In [1]:
from spike_detection import spikes_pipeline, PipelineConfig, pipeline, get_first_hydrophones

CENTER_FREQUENCY = 35000

LOWCUT = CENTER_FREQUENCY - 1000
HIGHCUT = CENTER_FREQUENCY + 1000

config = PipelineConfig(LOWCUT, HIGHCUT)
config.noise_floor_stdev_mult = 5.5

spikes = spikes_pipeline('../binaries/static/', config)

In [2]:
for group in spikes:
    print(len(group))

18
5
18
18


In [3]:
print(get_first_hydrophones(spikes, config))

[2, 0, 1, 0, 0, 2, 1, 1, 1, 2, 3, 0, 2, 2, 3, 0, 0, 3, 2, 3]


In [9]:
for spike_arr in spikes:
    for spike in spike_arr:
        print('<spike time="{:.6f}" amp="{:.6f}"> '.format(spike.time, spike.amplitude), end='')
    print()

<spike time="0.603051" amp="0.166969"> <spike time="2.651098" amp="0.417935"> <spike time="4.699232" amp="0.213092"> <spike time="6.747208" amp="0.276851"> <spike time="8.795211" amp="0.207666"> <spike time="10.843251" amp="0.310765"> <spike time="12.891342" amp="0.209022"> <spike time="14.939256" amp="0.219875"> <spike time="16.987301" amp="0.280921"> <spike time="19.035364" amp="0.359602"> 
<spike time="0.603010" amp="0.236719"> <spike time="2.651029" amp="0.249149"> <spike time="4.699062" amp="0.164898"> <spike time="6.747138" amp="0.242243"> <spike time="8.795114" amp="0.181472"> <spike time="10.843110" amp="0.152467"> <spike time="12.891214" amp="0.162135"> <spike time="14.939171" amp="0.213239"> <spike time="16.987217" amp="0.185615"> <spike time="19.035236" amp="0.191140"> 
<spike time="0.603174" amp="0.186961"> <spike time="2.651222" amp="0.170824"> <spike time="4.699112" amp="0.182927"> <spike time="6.747344" amp="0.153344"> <spike time="8.795219" amp="0.211165"> <spike time="

In [10]:
firsts = pipeline('./binaries/michaelholdingit/', config)

In [11]:
firsts

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]